## 安装依赖

In [2]:
%pip install openai

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 737.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 618.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 789.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 792.4 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 767.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 541.6 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0


## 配置

In [6]:
# 配置中包含敏感信息，发布时请删除！！
config = {
    'apikey': '',
    'basePath': '',
    'apikey1': 'pk-this-is-a-real-free-pool-token-for-everyone',
    'apikey2': 'pk-this-is-a-real-free-api-key-pk-for-everyone',
    'basePath1': 'https://ai.fakeopen.com/v1'
}

## 测试API
这里测试了三种api，分别是：
- 某第三方的转发api
- 公共的网页反代api
- 公共的网页反代api（120刀额度账号池）

结果如下：
- 第三方api：正常
- 公共api：会返回额外内容
- 公共api（120刀额度账号池）：不可用

### 第三方api

In [7]:
from openai import OpenAI
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=config["apikey"],
    base_url=config["basePath"],
)

In [8]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 .\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-8NyiWvLHvP2dBM7cVsY5sPKJlyWS9', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Here is the corrected data with the errors marked:\n\n[0, 0, 0, 1, 0]\n\nCorrected Data:\n1, New York, United States\n2, London, United Kingdom\n3, Paris, France\n4, Tokyo, Japan\n5, Sydney, Australia\n\nExplanation:\n- In the original data, the country for Tokyo was incorrectly mentioned as China. The correct country for Tokyo is Japan. Therefore, the result for that line is 1, indicating an error.\n- All other lines have the correct data, so the result for those lines is 0, indicating no errors.', role='assistant', function_call=None, tool_calls=None))], created=1700725632, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=126, prompt_tokens=167, total_tokens=293))
Here is the corrected data with the errors marked:

[0, 0, 0, 1, 0]

Corrected Data:
1, New York, United Stat

返回的内容是：
```markdown
[0, 0, 0, 1, 0]
Corrected Data: 
1, New York, United States
2, London, United Kingdom
3, Paris, France
4, Tokyo, Japan
5, Sydney, Australia
Explanation:
- In the original data, the country for Tokyo was incorrectly mentioned as China. The correct country for Tokyo is Japan. Therefore, the result for that line is 1, indicating an error.
- All other lines have the correct data, so the result for those lines is 0, indicating no errors.
```

In [9]:
# 在prompt中限制返回的结果格式
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 . Do not give any unnecessary imformation, return list only!\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-8NymJoUgIdaF9G03mzd3Vx9pDf3Ha', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='[1, 0, 0, 1, 0]', role='assistant', function_call=None, tool_calls=None))], created=1700725867, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=179, total_tokens=194))
[1, 0, 0, 1, 0]


添加了限制条件
```
 Do not give any unnecessary imformation, return list only!
 ```
 返回结果
```markdown
[0, 0, 0, 1, 0]
```

### 反代api

In [10]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=config["apikey1"],
    base_url=config["basePath1"],
)

In [11]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 .\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-qzA46cQPTFYzecl7ibleJRigge0AR', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Certainly, I can help you identify errors in the given data. Here\'s the corrected list format with the results:\n\n```python\ndata = [\n    "1,New York,United States",\n    "2,London,United Kingdom",\n    "3,Paris,France",\n    "4,Tokyo,Japan",  # Corrected country from China to Japan\n    "5,Sydney,Australia"\n]\n\n# Function to check errors and generate result list\ndef check_data(data):\n    results = []\n    for line in data:\n        columns = line.split(\',\')\n        if len(columns) == 3:\n            number, city, country = columns\n            # Check for errors in the data\n            if not number.isdigit() or not country.isalpha():\n                results.append(1)\n            else:\n                results.append(0)\n        else:\n            results.append(1)\n    return results\n\n# Get the result list\nresult_list = chec

返回结果：
```markdown
Certainly, I can help you identify errors in the given data. Here's the corrected list format with the results:

```python
data = [
    "1,New York,United States",
    "2,London,United Kingdom",
    "3,Paris,France",
    "4,Tokyo,Japan",  # Corrected country from China to Japan
    "5,Sydney,Australia"
]

# Function to check errors and generate result list
def check_data(data):
    results = []
    for line in data:
        columns = line.split(',')
        if len(columns) == 3:
            number, city, country = columns
            # Check for errors in the data
            if not number.isdigit() or not country.isalpha():
                results.append(1)
            else:
                results.append(0)
        else:
            results.append(1)
    return results

# Get the result list
result_list = check_data(data)

# Print the result list
print(result_list)
```

Output:
```python
[0, 0, 0, 1, 0]
```

In this example, the result list indicates errors with a value of 1 and correct entries with a value of 0. The error in the fourth line is due to the incorrect country (China should be Japan).
```

In [12]:
# 在prompt中限制返回的结果格式
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 . Do not give any unnecessary imformation, return list only!\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-D9TM82GwhhNZSGqavDcUU8wfimDDj', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='[0, 0, 0, 1, 0]', role='assistant', function_call=None, tool_calls=None))], created=1700726695, model='gpt-3.5-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=0, prompt_tokens=0, total_tokens=0))
[0, 0, 0, 1, 0]


添加了限制条件
```
 Do not give any unnecessary imformation, return list only!
 ```
 返回结果
```markdown
[0, 0, 0, 1, 0]
```

### 反代账号池

In [13]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=config["apikey2"],
    base_url=config["basePath1"],
)

In [18]:
# 在prompt中限制返回的结果格式
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 . Do not give any unnecessary imformation, return list only!\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'The OpenAI account associated with this API key has been deactivated. If you are the developer for this OpenAI app, please check your email for more information. If you are seeing this error while using another app or site, please reach out to them for more help.', 'type': 'invalid_request_error', 'param': None, 'code': 'account_deactivated'}}

反代的账号池已经不可用，大部分账号都被封禁了，返回结果如下：
```json
{'error': {'message': 'The OpenAI account associated with this API key has been deactivated. If you are the developer for this OpenAI app, please check your email for more information. If you are seeing this error while using another app or site, please reach out to them for more help.', 'type': 'invalid_request_error', 'param': None, 'code': 'account_deactivated'}}
```

## Error Detection

In [23]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=config["apikey"],
    base_url=config["basePath"],
)
# 在prompt中限制返回的结果格式
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system", 
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": "\
                You need to help find out errors in given data. The first column of data is number, The second column of data is city, the third column of data is the conuntry which the first column belongs to.\n\
                Desired Format: list format with the result, result has only two values: 0 or 1, when you find an error in a line,the result is 1 , otherwise the result is 0 . Do not give any unnecessary imformation, return list only!\n\
                Example output: [0,0,0,0,1]\n\
                Data:\n\
                1,New York,United States\n\
                2,London,United Kingdom\n\
                3,Paris,France\n\
                4,Tokyo,China\n\
                5,Sydney,Australia",
        }
    ],
    model="gpt-3.5-turbo",
    temperature=0,
)

print(chat_completion)
resp = chat_completion.choices[0].message.content

ChatCompletion(id='chatcmpl-8NzMRwwW4F0PE4KdWvHAtZB4o2519', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='[1, 0, 0, 1, 0]', role='assistant', function_call=None, tool_calls=None))], created=1700728107, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=15, prompt_tokens=179, total_tokens=194))


In [27]:
# 根据返回的结果，丢弃错误的数据
import ast
resp_list = ast.literal_eval(resp)
print(resp_list)
index = 1
for i in resp_list:
    if i == 1:
        print("丢弃数据：",index)
    index += 1

[1, 0, 0, 1, 0]
丢弃数据： 1
丢弃数据： 4


## 总结
GPT返回结果有随机性，即使temperature设置为0，也不一定能得到相同的结果。也许可以通过多次调用API，然后对结果进行统计，最后得到一个最终的结果。

反代API的随机性更大，因为相当于是在网页对话，无法自定义system prompt，也无法控制temperature等参数。

默认情况下，返回的结果中就会说明错误原因和修正结果，也许detect和repair可以合并?